# Merging and Joining DataFrames in Pandas

### What Are Merging and Joining DataFrames?

Merging and joining are ways to **combine two or more DataFrames** based on common columns or indexes, similar to SQL JOIN operations. This is a fundamental step when dealing with multiple datasets or splitting data into parts (like train/test splits).

For example, in the Titanic dataset, we might have passenger details in one table and ticket prices or survival info in another. Merging lets us combine these into a single, enriched DataFrame.

### Basic Merging Using `.merge()`

Pandas provides the `.merge()` method to join DataFrames based on one or more keys.

In [ ]:
import pandas as pd

df = pd.read_csv("data/train.csv")
df2 = pd.read_csv("data/ticket_fare.csv")  # hypothetical second dataset

merged = pd.merge(df, df2, on='Ticket')
print(merged.head())

- `on='Ticket'` means merge based on the `Ticket` column.
- By default, it performs an **inner join**, keeping rows with matching keys in both DataFrames.

### Types of Joins

- **Inner Join**: Only matching rows from both DataFrames.
- **Left Join**: All rows from left DataFrame + matching from right.
- **Right Join**: All rows from right DataFrame + matching from left.
- **Outer Join**: All rows from both DataFrames, filling missing values with NaN.

**Example**

In [ ]:
merged_left = pd.merge(df, df2, on='Ticket', how='left')

### Multiple Key Merging

We can merge on multiple columns:

In [ ]:
merged_multi = pd.merge(df, df2, on=['Ticket', 'Pclass'])

### Index-based Merging and `.join()`

- `.merge()` can also join on index using `left_index=True` and/or `right_index=True`.
- `.join()` is a simpler method for index-based joins.

**Example**

In [ ]:
merged_index = df.set_index('Ticket').join(df2.set_index('Ticket'), how='inner')

### Handling Overlapping Column Names (`suffixes` Parameter)

When two DataFrames have columns with the same name (other than the key), Pandas adds suffixes to distinguish them. By default, suffixes are `'_x'` and `'_y'`.

We can customize these:

In [ ]:
merged = pd.merge(df, df2, on='Ticket', suffixes=('_left', '_right'))
print(merged.columns)

This prevents confusion when both DataFrames have a column like `"Fare"`.

### Indicator Flag to Track Source of Rows (`indicator=True`)

Sometimes it's useful to know the origin of each row after merging (whether it was found in left only, right only, or both).

Use `indicator=True`:

In [ ]:
merged = pd.merge(df, df2, on='Ticket', how='outer', indicator=True)
print(merged['_merge'].value_counts())

The `_merge` column will show:

- `'left_only'` — row only in left DataFrame
- `'right_only'` — row only in right DataFrame
- `'both'` — row matched in both

### Performance Considerations with Large Datasets

- **Index our keys** before merging to speed up operations.
- Use `categorical` data types for keys with limited unique values.
- Avoid unnecessary columns before merging (select only needed columns).
- Consider chunked reading and merging for very large files.
- Use efficient join types (`inner` is faster than `outer`).

### Merging with SQL Databases or Other File Formats

Pandas supports reading from and writing to many data sources, making merging easy:

- Read from SQL databases using `pd.read_sql_query()` and then merge.
- Join data from CSV, Excel, JSON files by reading them as DataFrames.
- Use SQL joins inside databases before importing if data is very large.

**Example with SQL**

In [ ]:
import sqlite3

conn = sqlite3.connect('titanic.db')
df_sql = pd.read_sql_query("SELECT * FROM passenger_data", conn)
merged = pd.merge(df, df_sql, on='PassengerId', how='left')

### AI/ML Use Case: Data Enrichment and Feature Engineering

Merging allows combining raw features from multiple sources — such as passenger demographics, ticket info, and survival outcomes — to create rich datasets for modeling. Understanding join types helps prevent data leakage or missing data issues.

### Exercises

Q1. Merge Titanic train data with a hypothetical ticket fare dataset on `Ticket`.

In [ ]:
merged_q1 = pd.merge(df1, df2, on='Ticket')
print(merged_q1.head())

Q2. Perform a left join and observe missing values in the merged DataFrame.

In [ ]:
merged_q2 = pd.merge(df1, df2, on='Ticket', how='left')
print(merged_q2[['Ticket', 'Fare_Info']].isnull().sum())

Explanation: This keeps all rows from `df1`, and fills `NaN` in `Fare_Info` where there’s no match in `df2`.

Q3. Merge on multiple keys: `Ticket` and `Pclass`.

In [ ]:
merged_q3 = pd.merge(df1, df2, on=['Ticket', 'Pclass'], how='inner')
print(merged_q3.head())

Make sure both `Ticket` and `Pclass` exist in both DataFrames before merging.

Q4. Use the `suffixes` parameter to rename overlapping columns.

In [ ]:
df2_with_fare = df2.copy()
df2_with_fare['Fare'] = df2_with_fare['Fare_Info'] * 0.5  # Simulated overlapping column

merged_q4 = pd.merge(df1, df2_with_fare, on='Ticket', suffixes=('_original', '_info'))
print(merged_q4[['Fare_original', 'Fare_info']].head())

Q5. Use `indicator=True` to identify the origin of each row.

In [ ]:
merged_q5 = pd.merge(df1, df2, on='Ticket', how='outer', indicator=True)
print(merged_q5['_merge'].value_counts())
print(merged_q5[['Ticket', '_merge']].head())

Q6. Join two DataFrames using indexes instead of columns.

In [ ]:
df1_indexed = df1.set_index('Ticket')
df2_indexed = df2.set_index('Ticket')

joined_q6 = df1_indexed.join(df2_indexed, how='inner')
print(joined_q6.head())

This joins both DataFrames using their `Ticket` as index. We can also try `how='left'`, `right`, or `outer`.

### Summary

Merging and joining DataFrames is a critical skill for combining datasets, especially when working with real-world, multi-source data. Pandas provides flexible tools like `.merge()` and `.join()` to perform these operations easily, supporting SQL-like join types (inner, left, right, outer) and both column- and index-based joins.

Advanced features like customizing suffixes for overlapping columns, tracking merge origin with `indicator=True`, and performance tuning are important for large-scale, complex datasets. Additionally, Pandas integrates well with SQL databases and various file formats, allowing seamless data integration pipelines.

In AI and machine learning workflows, merging is essential for **feature engineering**, **data cleaning**, and **preparation**, helping to create comprehensive datasets that improve model accuracy and insights.